**Import necessary modules**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Load the dataset
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
#Drop the date and lights columns
df.drop(["date", "lights"], axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Appliances   19735 non-null  int64  
 1   T1           19735 non-null  float64
 2   RH_1         19735 non-null  float64
 3   T2           19735 non-null  float64
 4   RH_2         19735 non-null  float64
 5   T3           19735 non-null  float64
 6   RH_3         19735 non-null  float64
 7   T4           19735 non-null  float64
 8   RH_4         19735 non-null  float64
 9   T5           19735 non-null  float64
 10  RH_5         19735 non-null  float64
 11  T6           19735 non-null  float64
 12  RH_6         19735 non-null  float64
 13  T7           19735 non-null  float64
 14  RH_7         19735 non-null  float64
 15  T8           19735 non-null  float64
 16  RH_8         19735 non-null  float64
 17  T9           19735 non-null  float64
 18  RH_9         19735 non-null  float64
 19  T_ou

In [5]:
#Normalizing the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalized_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)


**Question 12**

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

In [6]:
#Select the features and target variable
target = normalized_df["T6"]
features = np.array(normalized_df["T2"]).reshape(-1,1)

#Split tghe dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=30, random_state=42)

In [7]:
#Model Training
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_test, y_test)

#Prediction from model
predictions = linear_model.predict(X_test)

In [8]:
from sklearn import metrics
r_2 = metrics.r2_score(y_test, predictions).round(2)
print(f" R_Squared: {r_2}")

 R_Squared: 0.48


**Question 13**

What is the Mean Absolute Error (in two decimal places)?

In [9]:
mae = metrics.mean_absolute_error(y_test, predictions).round(2)
print(f" MAE: {mae}")

 MAE: 0.09


**Question 14**

What is the Residual Sum of Squares (in two decimal places)?


In [10]:
rss = np.sum(np.square(y_test - predictions)).round(2)
print(f" RSS: {rss}")

 RSS: 0.32


**Question 15**

What is the Root Mean Squared Error (in three decimal places)?


In [11]:
mse = metrics.mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse).round(3)
print(f" RMSE: {rmse}")

 RMSE: 0.104


**Question 16**

What is the Coefficient of Determination (in two decimal places)? 

In [12]:
print(f"Coefficient: {linear_model.coef_.round(2)}")

Coefficient: [0.66]


**Question 17**

Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

**Comparing weights**

In [13]:
#Features and target selection
target_2 = normalized_df["Appliances"]
features_2 = normalized_df.iloc[:, 1:]

X_train, X_test, y_train, y_test = train_test_split(features_2, target_2, test_size=30, random_state=42)

In [14]:
# Train the linear model
linear_model_2 = LinearRegression()
linear_model_2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
#Train the Ridge model
from sklearn.linear_model import Ridge
ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [16]:
#train the lasso model
from sklearn.linear_model import Lasso
lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [17]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(linear_model_2, X_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, X_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.479502,-0.403472,-0.0
1,T_out,-0.307917,-0.222066,0.0
2,T2,-0.261161,-0.203011,0.0
3,T9,-0.198544,-0.196712,0.0
4,RH_8,-0.161369,-0.159552,-0.0
5,RH_out,-0.063819,-0.032599,-0.0
6,RH_7,-0.048498,-0.050779,-0.0
7,RH_9,-0.033971,-0.036581,-0.0
8,rv1,-0.000957,-0.000959,-0.0
9,rv2,-0.000957,-0.000959,-0.0


**Question 18**

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?



In [18]:
#Train the ridge model with no alpha value
ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)

#Obtain predictions
predictions_ridge = ridge_reg.predict(X_test)

#Evaluate model
mse = metrics.mean_squared_error(y_test, predictions_ridge)
rmse = np.sqrt(mse).round(3)
print(f" RMSE: {rmse}")

 RMSE: 0.055


In [19]:
#Train the Ridge model on alpha = 0.4
ridge_reg2 = Ridge(alpha=0.4)
ridge_reg2.fit(X_train, y_train)

#Obtain predictions
predictions_ridge1 = ridge_reg2.predict(X_test)

#Evaluate the model
mse = metrics.mean_squared_error(y_test, predictions_ridge1)
rmse = np.sqrt(mse).round(3)
print(f" RMSE: {rmse}")

 RMSE: 0.055


**Question 19**

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [20]:
#train the lasso model
from sklearn.linear_model import Lasso
lasso_reg2 = Lasso(alpha=0.001)
lasso_reg2.fit(X_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [21]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name]
  return weights_df


lasso_weights_df = get_weights_df(lasso_reg2, X_train, 'Lasso_weight')
print(lasso_weights_df)

       Features  Lasso_weight
0        RH_out     -0.050176
1            T1      0.000000
2     Tdewpoint      0.000000
3    Visibility      0.000000
4   Press_mm_hg     -0.000000
5         T_out      0.000000
6          RH_9     -0.000000
7            T9     -0.000000
8          RH_8     -0.000000
9            T8      0.000000
10         RH_7     -0.000000
11          rv1     -0.000000
12           T7     -0.000000
13           T6      0.000000
14         RH_5      0.000000
15           T5     -0.000000
16         RH_4      0.000000
17           T4     -0.000000
18         RH_3      0.000000
19           T3      0.000000
20         RH_2     -0.000000
21           T2      0.000000
22         RH_6     -0.000000
23          rv2     -0.000000
24    Windspeed      0.003287
25         RH_1      0.015491


**Question 20**

What is the new RMSE with the Lasso Regression (in 3 decimal places)?  

In [22]:
#Train the Ridge model on alpha = 0.4
lasso_reg2 = Lasso(alpha=0.4)
lasso_reg2.fit(X_train, y_train)

#Obtain predictions
predictions_lasso1 = lasso_reg2.predict(X_test)

#Evaluate the model
mse = metrics.mean_squared_error(y_test, predictions_lasso1)
rmse = np.sqrt(mse).round(3)
print(f" RMSE: {rmse}")


 RMSE: 0.055
